In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install genz-tokenize==1.2.7a2 -q
!pip install vncorenlp -q
!pip install pyvi -q

     |████████████████████████████████| 63.9 MB 137 kB/s 
     |████████████████████████████████| 2.6 MB 46.7 MB/s 
     |████████████████████████████████| 462 kB 73.2 MB/s 
     |████████████████████████████████| 8.5 MB 14.8 MB/s 
     |████████████████████████████████| 965 kB 61.3 MB/s 


In [3]:
from genz_tokenize import Tokenize
from genz_tokenize.preprocess import vncore_tokenize
import tensorflow as tf
import numpy as np
import math
from vncorenlp import VnCoreNLP
import json
import string
from typing import Dict, List, Optional, Union, Tuple
from pyvi import ViPosTagger, ViTokenizer

In [4]:
tokenizer = Tokenize()

In [5]:
vncore_jar = '/content/drive/MyDrive/Khóa luận tốt nghiệp/Paragraph_Raking/Tensorflow/VnCoreNLP/VnCoreNLP-1.1.1.jar'
vncore = VnCoreNLP(vncore_jar)

In [6]:
def remove_punc(text):
    punc = string.punctuation
    punc = punc.replace('_', '')
    return ''.join([i for i in text if i not in punc])

In [42]:
documents = [
             
    "Trách nhiệm của Phòng Tổ chức Hành chính. 1. Giải quyết các công việc hành chính có liên quan cho sinh viên; sao y các văn bằng, chứng chỉ của Trường. 2. Tiếp nhận và phân phối bưu phẩm, thư cho sinh viên.",
    "Sinh viên bị kỷ luật mức buộc thôi học không được đánh giá kết quả rèn luyện.",
    "Sinh viên không tự đánh giá kết quả rèn luyện theo quy định học kỳ nào sẽ bị nhận điểm 0 trong học kỳ đó.",
    "Quy chế đào tạo theo hệ thống tín chỉ là tập hợp những quy định về phương thức đào tạo thực hiện theo hình thức tích lũy tín chỉ; trong đó sinh viên chủ động lựa chọn học từng học phần (tuân theo một số ràng buộc được quy định trước) nhắm tích lũy từng phần kiến thức và tiến tới hoàn thành toàn bộ chương trình đào tạo để được cấp văn bằng tốt nghiệp.",
    "Sinh viên nghỉ học tạm thời được bảo lưu kết quả rèn luyện sẽ được đánh giá kết quả rèn luyện khi tiếp tục trở lại học tập theo quy định.",
    "Kết quả rèn luyện được phân thành các loại: Xuất sắc, tốt, khá, trung bình, yếu và kém. Phân loại kết quả rèn luyện: Từ 90 đến 100 điểm đạt loại xuất sắc; Từ 80 đến dưới 90 đạt loại tốt; Từ 65 đến dưới 80 đạt loại khá; Từ 50 đến dưới 65 đạt loại trung bình; Từ 35 đến dưới 50 đạt loại yếu; Dưới 35 điểm đạt loại kém.",

]
query = "Không biết trách nhiệm của phòng tổ chức hành chính là gì ạ? Em cám ơn ạ"

# query = "đào tạo theo tín chỉ là gì"

In [43]:
# ViPosTagger.postagging(ViTokenizer.tokenize(query))

In [44]:
documents = [
            remove_punc(vncore_tokenize(i, vncore)).lower()
            for i in documents
]
query = remove_punc(vncore_tokenize(query, vncore)).lower()

In [45]:
documents

['sinh_viên chuyển trường được sự đồng_ý của hiệu_trưởng hai cơ_sở giáo_dục đại_học thì được bảo_lưu kết_quả rèn_luyện của cơ_sở giáo_dục đại_học cũ khi học tại cơ_sở giáo_dục đại_học mới và tiếp_tục được đánh_giá kết_quả rèn_luyện ở các học_kỳ tiếp_theo ',
 'sinh_viên bị kỷ_luật mức buộc thôi học không được đánh_giá kết_quả rèn_luyện ',
 'sinh_viên không tự đánh_giá kết_quả rèn_luyện theo quy_định học_kỳ nào sẽ bị nhận điểm 0 trong học_kỳ đó ',
 'quy_chế đào_tạo theo hệ_thống tín_chỉ là tập_hợp những quy_định về phương_thức đào_tạo thực_hiện theo hình_thức tích_luỹ tín_chỉ  trong đó sinh_viên chủ_động lựa_chọn học từng học_phần  tuân theo một_số ràng_buộc được quy_định trước  nhắm tích_luỹ từng phần kiến_thức và tiến tới hoàn_thành toàn_bộ chương_trình đào_tạo để được cấp văn_bằng tốt_nghiệp ',
 'sinh_viên nghỉ học tạm_thời được bảo_lưu kết_quả rèn_luyện sẽ được đánh_giá kết_quả rèn_luyện khi tiếp_tục trở_lại học_tập theo quy_định ',
 'kết_quả rèn_luyện được phân thành các loại  xuất_

In [46]:
query

'học lại là gì'

In [47]:
label = ['ai', 'cai gi', 'con vat', 'nhu the nao', 'number', 'tai sao', 'thoi gian', 'time', 'thuc vat', 'yes no', 'location']
postag_label = ['B', 'Np', 'Nc', 'Nu', 'N', 'Ny', 'Ni', 'Nb', 'V', 'Vb', 'A', 'Ab', 'P', 'R', 'L', 'M', 'E', 'C', 'Cc', 'I', 'T', 'Y', 'Z', 'X', 'CH']

def getTypeOfWord(vncore, text):
    out = np.zeros(shape=(277, len(postag_label)), dtype = np.int64)
    pos = pos_tag(text, vncore)
    for i in range(len(pos)):
        if i == 277:
            print(pos)
        index_pos = postag_label.index(pos[i][1])
        out[i, index_pos] = 1
    return out

def pos_tag(text, vncore):
    text = text.replace('_', ' ')
    tag = vncore.pos_tag(text)
    combine = []
    for i in tag:
        combine += i
    return combine

In [48]:
# tag = vncore.pos_tag("mục đích của công tác sinh viên là gì?")

# weight

In [49]:
label = ['ai', 'cai gi', 'con vat', 'nhu the nao', 'number', 'tai sao', 'thoi gian', 'time', 'thuc vat', 'yes no', 'location']
postag_label = ['B', 'Np', 'Nc', 'Nu', 'N', 'Ny', 'Ni', 'Nb', 'V', 'Vb', 'A', 'Ab', 'P', 'R', 'L', 'M', 'E', 'C', 'Cc', 'I', 'T', 'Y', 'Z', 'X', 'CH']

class Embedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, dim,max_position_embeddings,  **kwargs):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=1e-12, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(rate=0.1)

    def build(self, input_shape: tf.TensorShape):
        with tf.name_scope("word_embeddings"):
            self.weight = self.add_weight(
                name="weight",
                shape=[self.vocab_size, self.dim],
            )

        with tf.name_scope("position_embeddings"):
            self.position_embeddings = self.add_weight(
                name="embeddings",
                shape=[self.max_position_embeddings, self.dim],
            )
        super().build(input_shape)

    def call(self, input_ids=None):
        inputs_embeds = tf.gather(params=self.weight, indices=input_ids)
        input_shape = tf.shape(input_ids)
        position_ids = tf.expand_dims(tf.range(start=0, limit=input_shape[-1]), axis=0)
        position_embeds = tf.gather(params=self.position_embeddings, indices=position_ids)
        final_embeddings = inputs_embeds + position_embeds
        final_embeddings = self.LayerNorm(inputs=final_embeddings)
        final_embeddings = self.dropout(inputs=final_embeddings)
        return final_embeddings

class FeedFoward(tf.keras.layers.Layer):
    def __init__(self, d_model, num_class):
        super(FeedFoward, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(d_model)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.out = tf.keras.layers.Dense(num_class, activation = 'softmax')
    
    def call(self, inputs):
        inputs = self.flatten(inputs)
        x = self.fc1(inputs)
        x = self.dropout(x)
        return self.out(x)
class ClassifyModel(tf.keras.Model):
    def __init__(self, vocab_size,max_position_embeddings, d_model, num_heads, num_class):
        super(ClassifyModel, self).__init__()
        self.embedding = Embedding(vocab_size, d_model, max_position_embeddings)
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads, d_model, dropout = 0.1)
        self.fc = tf.keras.layers.Dense(1)
        self.fw = FeedFoward(d_model, num_class)
    def call(self, inputs):
        x = self.embedding(inputs)
        mask = self.mask(inputs)
        out_mha = self.mha(x, x, x)
        x = self.fc(self.layernorm(x+out_mha))
        return self.fw(x)

    def mask(self, inputs):
        masks = tf.logical_not(tf.math.equal(inputs, 0))
        masks = tf.cast(masks, dtype = tf.float32)
        return masks[:, tf.newaxis, tf.newaxis, :]

model_cls = ClassifyModel(tokenizer.vocab_size(), 1000, 256, 3, len(label))

def fullConnected(d_model):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(d_model*2, activation='gelu'))
    model.add(tf.keras.layers.Dense(d_model, activation = 'sigmoid'))
    return model

class QuestionAnalys(tf.keras.Model):
    def __init__(self, d_model, maxlen, model_cls, num_heads):
        super(QuestionAnalys, self).__init__()
        self.model_cls = model_cls
        self.model_cls.trainable = False

        self.mha1 = tf.keras.layers.MultiHeadAttention(num_heads, d_model)
        self.mha2 = tf.keras.layers.MultiHeadAttention(num_heads, d_model)

        self.layerNorm1 = tf.keras.layers.LayerNormalization()
        self.layerNorm2 = tf.keras.layers.LayerNormalization()

        self.fc1 = fullConnected(d_model)
        self.fc2 = fullConnected(d_model)
        self.fc3 = fullConnected(d_model)

        self.fc_postag = tf.keras.layers.Dense(d_model)
        self.fc_type = tf.keras.layers.Dense(d_model)

        self.out1 = tf.keras.layers.Dense(1, activation = 'gelu')
        self.flatten = tf.keras.layers.Flatten()
        self.out2 = tf.keras.layers.Dense(d_model, activation = 'gelu')
        self.out3 = tf.keras.layers.Dense(maxlen)

        self.dropout1 = tf.keras.layers.Dropout(0.2)
        self.dropout2= tf.keras.layers.Dropout(0.2)

    def mask(self, inputs):
        masks = tf.logical_not(tf.math.equal(inputs, 0))
        masks = tf.cast(masks, dtype = tf.float32)
        return masks[:, tf.newaxis, tf.newaxis, :]
    
    def call(self, x):
        inputs, input_postag = x
        emb = self.model_cls.embedding(inputs)
        mask = self.mask(inputs)
        out_mha = self.mha1(emb, emb, emb, attention_mask = mask)
        norm = self.layerNorm1(out_mha+emb)
        ff = self.fc1(self.dropout1(norm))

        postag = self.fc_postag(input_postag)
        out_mha = self.mha2(ff, postag, ff, attention_mask = mask)
        norm = self.layerNorm2(ff+out_mha)
        ff = self.fc2(self.dropout2(norm))

        inputs = tf.cast(inputs, dtype = tf.int32)
        out_type = tf.expand_dims(self.fc_type(self.model_cls(inputs)), axis = 1)
        out = tf.concat([ff, out_type], axis = 1)
        out = self.fc3(out)
        out = self.out1(out)
        out = self.flatten(out)
        out = self.out2(out)
        out = self.out3(out)
        return out
    
    def compile(self, loss, optimizer):
        super(QuestionAnalys, self).compile()
        self.optimizer = optimizer
        self.loss = loss
        self.train_loss = tf.keras.metrics.Mean()
        self.test_loss = tf.keras.metrics.Mean()
        
    @property
    def metrics(self):
        return [self.train_loss, self.test_loss]

    def train_step(self, data):
        x1, x2, y = data
        with tf.GradientTape() as tape:
            predicts = self([x1, x2])
            l = self.loss(y, predicts)
        gradient = tape.gradient(l, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradient, self.trainable_variables))
        self.train_loss(l)
        return {'loss': self.train_loss.result()}
        
    def test_step(self, data):
        x1, x2, y = data
        predicts = self([x1, x2])
        l = self.loss(y, predicts)
        self.test_loss(l)
        return {'loss': self.test_loss.result()}

d_model = 256
num_heads = 6
model_analys = QuestionAnalys(d_model, 277, model_cls, num_heads)
checkpoint = tf.train.Checkpoint(
        model=model_analys)
ckpt_manager = tf.train.CheckpointManager(
    checkpoint, '/content/drive/MyDrive/Khóa luận tốt nghiệp/weight_of_word/chekcpoint2', max_to_keep=5)
if ckpt_manager.latest_checkpoint:
    checkpoint.restore(ckpt_manager.latest_checkpoint)
    print('\nLatest checkpoint restored!!!\n')


Latest checkpoint restored!!!



In [50]:
input_ids = tokenizer(query, max_len = 277, padding = True, truncation = True)['input_ids']
input_ids = np.expand_dims(input_ids, axis = 0)
postag= np.expand_dims(getTypeOfWord(vncore, query), axis = 0)

In [51]:
pred_type = model_cls(input_ids)[0]
print(label[np.argmax(pred_type)])

cai gi


In [52]:
w = {}
pred = model_analys([input_ids, postag])[0]
for i, v in enumerate(query.split()):
    w[v] = pred[i]

In [53]:
for k, v in w.items():
    print('%s ----------> %.2f'%(k, v))

học ----------> 0.85
lại ----------> 0.39
là ----------> 0.23
gì ----------> 0.28


# Ranking

In [54]:
class BM25W:
    def __init__(self, model, documents, b: float = 0.75, k1: float = 1.2) -> None:
        self.b = b
        self.k1 = k1
        self.model = model

        self.num_doc = 0
        self.fieldLens = []
        self.frequency_word_in_doc = []
        self.documents = []

        for document in documents:
            frequency = {}
            document = document.lower().split()
            self.documents.append(document)
            self.num_doc += 1
            self.fieldLens.append(len(document))
            for i in document:
                try:
                    frequency[i] += 1
                except:
                    frequency[i] = 1
            self.frequency_word_in_doc.append(frequency)
        self.avgFieldLen = np.mean(self.fieldLens)

    def cal_w(self, q):
        input_ids = tokenizer(q, max_len = 277, padding = True, truncation = True)['input_ids']
        input_ids = np.expand_dims(input_ids, axis = 0)
        postag= np.expand_dims(getTypeOfWord(vncore, q), axis = 0)
        self.w = {}
        pred = self.model([input_ids, postag])[0]
        for i, v in enumerate(q.split()):
            self.w[v] = pred[i]
    def cal_idf(self, q: str) -> float:
        f_q = sum([1 if q in i else 0 for i in self.documents])
        return np.log(1+(len(self.documents)-f_q+0.5+self.w[q])/(f_q+0.5))

    def get_score(self, query: str):
        query = query.lower()
        self.cal_w(query)
        query = query.split()
        scores = []
        for i, doc in enumerate(self.documents):
            score = 0
            for q in query:
                f = self.frequency_word_in_doc[i].get(q, 0)
                idf = self.cal_idf(q)
                score += idf*((f*(self.k1+1+self.w[q]))/(f+self.k1 *
                              (1-self.b+self.b*(len(doc)/self.avgFieldLen))))
            scores.append(score)
        return scores

In [55]:
bm25 = BM25W(model_analys, documents = documents)
scores = bm25.get_score(query)
for i in range(len(documents)):
    print('doc %d: score: %.2f'%(i+1, scores[i].numpy()))

doc 1: score: 0.72
doc 2: score: 1.05
doc 3: score: 0.00
doc 4: score: 2.11
doc 5: score: 0.94
doc 6: score: 0.00


# Question Answer

In [56]:
class Config:
    def __init__(
        self,
        input_vocab_size = 50265,
        target_vocab_size = 40000,
        max_position_embeddings=258,
        hidden_size = 256,
        initializer_range = 0.02,
        layer_norm_eps = 1e-6,
        hidden_dropout_prob = 0.1,
        attention_probs_dropout_prob = 0.1,
        num_attention_heads = 8,
        num_hidden_layers = 8,
        is_decoder = True,
        intermediate_size = 1024,
        type_vocab_size = 1
    ):
        self.input_vocab_size = input_vocab_size
        self.target_vocab_size = target_vocab_size
        self.max_position_embeddings = max_position_embeddings
        self.hidden_size = hidden_size
        self.initializer_range = initializer_range
        self.layer_norm_eps = layer_norm_eps
        self.hidden_dropout_prob = hidden_dropout_prob
        self.num_attention_heads = num_attention_heads
        self.num_hidden_layers = num_hidden_layers
        self.is_decoder = is_decoder
        self.intermediate_size = intermediate_size
        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.type_vocab_size = type_vocab_size
        

In [57]:
class TFRobertaEmbeddings(tf.keras.layers.Layer):
    """
    Same as BertEmbeddings with a tiny tweak for positional embeddings indexing.
    """

    def __init__(self, config, vocab_size, **kwargs):
        super().__init__(**kwargs)

        self.padding_idx = 0
        self.vocab_size = vocab_size
        self.type_vocab_size = config.type_vocab_size
        self.hidden_size = config.hidden_size
        self.max_position_embeddings = config.max_position_embeddings
        self.initializer_range = config.initializer_range
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(rate=config.hidden_dropout_prob)

    def build(self, input_shape: tf.TensorShape):
        with tf.name_scope("word_embeddings"):
            self.weight = self.add_weight(
                name="weight",
                shape=[self.vocab_size, self.hidden_size],
                initializer=get_initializer(self.initializer_range),
            )

        with tf.name_scope("token_type_embeddings"):
            self.token_type_embeddings = self.add_weight(
                name="token_embeddings",
                shape=[self.type_vocab_size, self.hidden_size],
                initializer=get_initializer(self.initializer_range),
            )

        with tf.name_scope("position_embeddings"):
            self.position_embeddings = self.add_weight(
                name="embeddings",
                shape=[self.max_position_embeddings, self.hidden_size],
                initializer=get_initializer(self.initializer_range),
            )

        super().build(input_shape)

    def create_position_ids_from_input_ids(self, input_ids, past_key_values_length=0):
        mask = tf.cast(tf.math.not_equal(input_ids, self.padding_idx), dtype=input_ids.dtype)
        incremental_indices = (tf.math.cumsum(mask, axis=1) + past_key_values_length) * mask

        return incremental_indices + self.padding_idx

    def call(
        self,
        input_ids=None,
        position_ids=None,
        token_type_ids=None,
        inputs_embeds=None,
        past_key_values_length=0,
        training=False,
    ):
        """
        Applies embedding based on inputs tensor.
        Returns:
            final_embeddings (`tf.Tensor`): output embedding tensor.
        """
        assert not (input_ids is None and inputs_embeds is None)

        if input_ids is not None:
            inputs_embeds = tf.gather(params=self.weight, indices=input_ids)

        input_shape = shape_list(inputs_embeds)[:-1]

        if token_type_ids is None:
            token_type_ids = tf.fill(dims=input_shape, value=0)

        if position_ids is None:
            if input_ids is not None:
                # Create the position ids from the input token ids. Any padded tokens remain padded.
                position_ids = self.create_position_ids_from_input_ids(
                    input_ids=input_ids, past_key_values_length=past_key_values_length
                )
            else:
                position_ids = tf.expand_dims(
                    tf.range(start=self.padding_idx + 1, limit=input_shape[-1] + self.padding_idx + 1), axis=0
                )

        position_embeds = tf.gather(params=self.position_embeddings, indices=position_ids)
        token_type_embeds = tf.gather(params=self.token_type_embeddings, indices=token_type_ids)
        final_embeddings = inputs_embeds + position_embeds + token_type_embeds
        final_embeddings = self.LayerNorm(inputs=final_embeddings)
        final_embeddings = self.dropout(inputs=final_embeddings, training=training)

        return final_embeddings

In [58]:
def get_initializer(initializer_range: float = 0.02) -> tf.initializers.TruncatedNormal:
    return tf.keras.initializers.TruncatedNormal(stddev=initializer_range)

In [59]:
def shape_list(tensor: Union[tf.Tensor, np.ndarray]) -> List[int]:
    if isinstance(tensor, np.ndarray):
        return list(tensor.shape)

    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

In [60]:
class TFRobertaSelfAttention(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)

        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                f"The hidden size ({config.hidden_size}) is not a multiple of the number "
                f"of attention heads ({config.num_attention_heads})"
            )

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size
        self.sqrt_att_head_size = math.sqrt(self.attention_head_size)

        self.query = tf.keras.layers.Dense(
            units=self.all_head_size, kernel_initializer=get_initializer(config.initializer_range), name="query"
        )
        self.key = tf.keras.layers.Dense(
            units=self.all_head_size, kernel_initializer=get_initializer(config.initializer_range), name="key"
        )
        self.value = tf.keras.layers.Dense(
            units=self.all_head_size, kernel_initializer=get_initializer(config.initializer_range), name="value"
        )
        self.dropout = tf.keras.layers.Dropout(rate=config.attention_probs_dropout_prob)

        self.is_decoder = config.is_decoder

    def transpose_for_scores(self, tensor: tf.Tensor, batch_size: int) -> tf.Tensor:
        # Reshape from [batch_size, seq_length, all_head_size] to [batch_size, seq_length, num_attention_heads, attention_head_size]
        tensor = tf.reshape(tensor=tensor, shape=(batch_size, -1, self.num_attention_heads, self.attention_head_size))

        # Transpose the tensor from [batch_size, seq_length, num_attention_heads, attention_head_size] to [batch_size, num_attention_heads, seq_length, attention_head_size]
        return tf.transpose(tensor, perm=[0, 2, 1, 3])

    def call(
        self,
        hidden_states: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        encoder_hidden_states: tf.Tensor,
        encoder_attention_mask: tf.Tensor,
        past_key_value: Tuple[tf.Tensor],
        output_attentions: bool,
        training: bool = False,
    ) -> Tuple[tf.Tensor]:
        batch_size = shape_list(hidden_states)[0]
        mixed_query_layer = self.query(inputs=hidden_states)

        # If this is instantiated as a cross-attention module, the keys
        # and values come from an encoder; the attention mask needs to be
        # such that the encoder's padding tokens are not attended to.
        is_cross_attention = encoder_hidden_states is not None

        if is_cross_attention and past_key_value is not None:
            # reuse k,v, cross_attentions
            key_layer = past_key_value[0]
            value_layer = past_key_value[1]
            attention_mask = encoder_attention_mask
        elif is_cross_attention:
            key_layer = self.transpose_for_scores(self.key(inputs=encoder_hidden_states), batch_size)
            value_layer = self.transpose_for_scores(self.value(inputs=encoder_hidden_states), batch_size)
            attention_mask = encoder_attention_mask
        elif past_key_value is not None:
            key_layer = self.transpose_for_scores(self.key(inputs=hidden_states), batch_size)
            value_layer = self.transpose_for_scores(self.value(inputs=hidden_states), batch_size)
            key_layer = tf.concat([past_key_value[0], key_layer], axis=2)
            value_layer = tf.concat([past_key_value[1], value_layer], axis=2)
        else:
            key_layer = self.transpose_for_scores(self.key(inputs=hidden_states), batch_size)
            value_layer = self.transpose_for_scores(self.value(inputs=hidden_states), batch_size)

        query_layer = self.transpose_for_scores(mixed_query_layer, batch_size)

        if self.is_decoder:
            past_key_value = (key_layer, value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        # (batch size, num_heads, seq_len_q, seq_len_k)
        attention_scores = tf.matmul(query_layer, key_layer, transpose_b=True)
        dk = tf.cast(self.sqrt_att_head_size, dtype=attention_scores.dtype)
        attention_scores = tf.divide(attention_scores, dk)

        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in TFRobertaModel call() function)
            attention_scores = tf.add(attention_scores, attention_mask)

        # Normalize the attention scores to probabilities.
        attention_probs = tf.nn.softmax(logits=attention_scores, axis=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(inputs=attention_probs, training=training)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = tf.multiply(attention_probs, head_mask)

        attention_output = tf.matmul(attention_probs, value_layer)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])

        # (batch_size, seq_len_q, all_head_size)
        attention_output = tf.reshape(tensor=attention_output, shape=(batch_size, -1, self.all_head_size))
        outputs = (attention_output, attention_probs) if output_attentions else (attention_output,)

        if self.is_decoder:
            outputs = outputs + (past_key_value,)
        return outputs # tuple

In [61]:
class TFRobertaSelfOutput(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.hidden_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(rate=config.hidden_dropout_prob)

    def call(self, hidden_states: tf.Tensor, input_tensor: tf.Tensor, training: bool = False) -> tf.Tensor:
        hidden_states = self.dense(inputs=hidden_states)
        hidden_states = self.dropout(inputs=hidden_states, training=training)
        hidden_states = self.LayerNorm(inputs=hidden_states + input_tensor)

        return hidden_states

In [62]:
class TFRobertaAttention(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)

        self.self_attention = TFRobertaSelfAttention(config, name="self")
        self.dense_output = TFRobertaSelfOutput(config, name="output")

    def call(
        self,
        input_tensor: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        encoder_hidden_states: tf.Tensor,
        encoder_attention_mask: tf.Tensor,
        past_key_value: Tuple[tf.Tensor],
        output_attentions: bool,
        training: bool = False,
    ) -> Tuple[tf.Tensor]:
        self_outputs = self.self_attention(
            hidden_states=input_tensor,
            attention_mask=attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            past_key_value=past_key_value,
            output_attentions=output_attentions,
            training=training,
        )
        attention_output = self.dense_output(
            hidden_states=self_outputs[0], input_tensor=input_tensor, training=training
        )
        # add attentions (possibly with past_key_value) if we output them
        outputs = (attention_output,) + self_outputs[1:]

        return outputs

In [63]:
class TFRobertaIntermediate(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.intermediate_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.intermediate_act_fn = tf.nn.gelu

    def call(self, hidden_states: tf.Tensor) -> tf.Tensor:
        hidden_states = self.dense(inputs=hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)

        return hidden_states


In [64]:
class TFRobertaOutput(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.hidden_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(rate=config.hidden_dropout_prob)

    def call(self, hidden_states: tf.Tensor, input_tensor: tf.Tensor, training: bool = False) -> tf.Tensor:
        hidden_states = self.dense(inputs=hidden_states)
        hidden_states = self.dropout(inputs=hidden_states, training=training)
        hidden_states = self.LayerNorm(inputs=hidden_states + input_tensor)

        return hidden_states

In [65]:
class TFRobertaEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)

        self.attention = TFRobertaAttention(config, name="attention")
        self.intermediate = TFRobertaIntermediate(config, name="intermediate")
        self.bert_output = TFRobertaOutput(config, name="output")

    def call(
        self,
        hidden_states: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        past_key_value: Optional[Tuple[tf.Tensor]],
        output_attentions: bool,
        training: bool = False,
    ) -> Tuple[tf.Tensor]:
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attn_past_key_value = past_key_value[:2] if past_key_value is not None else None
        self_attention_outputs = self.attention(
            input_tensor=hidden_states,
            attention_mask=attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=None,
            encoder_attention_mask=None,
            past_key_value=self_attn_past_key_value,
            output_attentions=output_attentions,
            training=training,
        )
        attention_output = self_attention_outputs[0]

        outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights

        intermediate_output = self.intermediate(hidden_states=attention_output)
        layer_output = self.bert_output(
            hidden_states=intermediate_output, input_tensor=attention_output, training=training
        )
        outputs = (layer_output,) + outputs  # add attentions if we output them
        return outputs

In [66]:
class TFRobertaEncoder(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.layer = [TFRobertaEncoderLayer(config, name=f"layer_._{i}") for i in range(config.num_hidden_layers)]
        
    def call(
        self,
        hidden_states: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        past_key_values: Optional[Tuple[Tuple[tf.Tensor]]],
        training: bool = False,
    ) -> tf.Tensor:
        attention_mask_shape = shape_list(attention_mask)
        extended_attention_mask = tf.reshape(
                attention_mask, (attention_mask_shape[0], 1, 1, attention_mask_shape[1])
        )
        extended_attention_mask = tf.cast(extended_attention_mask, dtype=hidden_states.dtype)
        one_cst = tf.constant(1.0, dtype=hidden_states.dtype)
        ten_thousand_cst = tf.constant(-10000.0, dtype=hidden_states.dtype)
        extended_attention_mask = tf.multiply(tf.subtract(one_cst, extended_attention_mask), ten_thousand_cst)

        for i, layer_module in enumerate(self.layer):

            past_key_value = past_key_values[i] if past_key_values is not None else None

            layer_outputs = layer_module(
                hidden_states=hidden_states,
                attention_mask=extended_attention_mask,
                head_mask=head_mask[i],
                past_key_value=past_key_value,
                output_attentions=True,
                training=training,
            )
            hidden_states = layer_outputs[0]

        return hidden_states
    

In [67]:
class TFRobertaDecoderLayer(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)

        self.attention = TFRobertaAttention(config, name="attention")

        self.crossattention = TFRobertaAttention(config, name="crossattention")
        self.intermediate = TFRobertaIntermediate(config, name="intermediate")
        self.bert_output = TFRobertaOutput(config, name="output")

    def call(
        self,
        hidden_states: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        encoder_hidden_states: Optional[tf.Tensor],
        encoder_attention_mask: Optional[tf.Tensor],
        past_key_value: Optional[Tuple[tf.Tensor]],
        output_attentions: bool,
        training: bool = False,
    ) -> tf.Tensor:
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attn_past_key_value = past_key_value[:2] if past_key_value is not None else None
        
        self_attention_outputs = self.attention(
            input_tensor=hidden_states,
            attention_mask=attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=None,
            encoder_attention_mask=None,
            past_key_value=self_attn_past_key_value,
            output_attentions=output_attentions,
            training=training,
        )
        attention_output = self_attention_outputs[0]

        # if decoder, the last output is tuple of self-attn cache
        outputs = self_attention_outputs[1:-1]
        present_key_value = self_attention_outputs[-1]

        cross_attn_present_key_value = None
        # cross_attn cached key/values tuple is at positions 3,4 of past_key_value tuple
        cross_attn_past_key_value = past_key_value[-2:] if past_key_value is not None else None
        cross_attention_outputs = self.crossattention(
            input_tensor=attention_output,
            attention_mask=attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            past_key_value=cross_attn_past_key_value,
            output_attentions=output_attentions,
            training=training,
        )
        attention_output = cross_attention_outputs[0]
        outputs = outputs + cross_attention_outputs[1:-1]  # add cross attentions if we output attention weights

        # add cross-attn cache to positions 3,4 of present_key_value tuple
        cross_attn_present_key_value = cross_attention_outputs[-1]
        present_key_value = present_key_value + cross_attn_present_key_value

        intermediate_output = self.intermediate(hidden_states=attention_output)
        layer_output = self.bert_output(
            hidden_states=intermediate_output, input_tensor=attention_output, training=training
        )
        outputs = (layer_output,) + outputs  # add attentions if we output them

        # if decoder, return the attn key/values as the last output
        outputs = outputs + (present_key_value,)
        return outputs

In [68]:
class TFRobertaDecoder(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.layer = [TFRobertaDecoderLayer(config, name=f"layer_._{i}") for i in range(config.num_hidden_layers)]

    def call(
        self,
        hidden_states: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        encoder_hidden_states: Optional[tf.Tensor],
        encoder_attention_mask: Optional[tf.Tensor],
        past_key_values: Optional[Tuple[Tuple[tf.Tensor]]],
        training: bool = False,
    ) -> tf.Tensor:
        attention_mask_shape = shape_list(attention_mask)
        input_shape = shape_list(hidden_states)
        batch_size, seq_length, _ = input_shape
        mask_seq_length = seq_length
        seq_ids = tf.range(mask_seq_length)

        causal_mask = tf.less_equal(
                tf.tile(seq_ids[None, None, :], (batch_size, mask_seq_length, 1)),
                seq_ids[None, :, None],
        )

        causal_mask = tf.cast(causal_mask, dtype=attention_mask.dtype)
        extended_attention_mask = causal_mask * attention_mask[:, None, :]
        attention_mask_shape = shape_list(extended_attention_mask)
        extended_attention_mask = tf.reshape(
            extended_attention_mask, (attention_mask_shape[0], 1, attention_mask_shape[1], attention_mask_shape[2])
        )

        extended_attention_mask = tf.cast(extended_attention_mask, dtype=hidden_states.dtype)
        one_cst = tf.constant(1.0, dtype=hidden_states.dtype)
        ten_thousand_cst = tf.constant(-10000.0, dtype=hidden_states.dtype)
        extended_attention_mask = tf.multiply(tf.subtract(one_cst, extended_attention_mask), ten_thousand_cst)

        encoder_attention_mask = tf.cast(encoder_attention_mask, dtype=extended_attention_mask.dtype)
        encoder_extended_attention_mask = encoder_attention_mask[:, None, None, :]
        
        encoder_extended_attention_mask = (1.0 - encoder_extended_attention_mask) * -10000.0
        
        for i, layer_module in enumerate(self.layer):

            past_key_value = past_key_values[i] if past_key_values is not None else None

            layer_outputs = layer_module(
                hidden_states=hidden_states,
                attention_mask=extended_attention_mask,
                head_mask=head_mask[i],
                encoder_hidden_states=encoder_hidden_states,
                encoder_attention_mask=encoder_extended_attention_mask,
                past_key_value=past_key_value,
                output_attentions=True,
                training=training,
            )
            hidden_states = layer_outputs[0]
        return hidden_states

In [69]:
class TFRobertaModel(tf.keras.Model):
    def __init__(self, config, model_weight, maxlen_c, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.model_weight = model_weight
        self.model_weight.trainable = False
        self.embedding_decoder = TFRobertaEmbeddings(config, config.target_vocab_size, name = 'embedding_decoder')
        self.encoder = TFRobertaEncoder(config, name = 'encoder')
        self.decoder = TFRobertaDecoder(config, name = 'decoder')

        self.logit = tf.keras.layers.Dense(2)
        self.start = tf.keras.layers.Dense(maxlen_c)
        self.end = tf.keras.layers.Dense(maxlen_c)
    
    def call(self, 
            encoder_inputs: Optional[tf.Tensor],
            encoder_attention_mask: Optional[tf.Tensor],
            postag: Optional[tf.Tensor],
            encoder_token_type_id: tf.Tensor,
            head_mask: tf.Tensor,
            decoder_inputs: tf.Tensor,
            decoder_attention_mask: tf.Tensor,
            decoder_token_type_id: tf.Tensor,
            training: bool = False):
        if head_mask is None:
            head_mask = [None] * self.config.num_hidden_layers
        if encoder_token_type_id is None:
            input_shape = shape_list(encoder_inputs)
            encoder_token_type_id = tf.fill(dims=input_shape, value=0)
        
        # input_ww = tf.pad(encoder_inputs, [[0, 0], [0, 177]], 'CONSTANT', constant_values=0)
        ww = self.model_weight([encoder_inputs, postag])
        ww = tf.expand_dims(ww, axis = -1)
        ww = tf.repeat(ww, self.config.hidden_size, axis=-1)
        hidden_states = self.model_weight.model_cls.embedding(encoder_inputs)*ww
        encoder_output = self.encoder(
                                    hidden_states = hidden_states,
                                    attention_mask = encoder_attention_mask,
                                    head_mask = head_mask,
                                    past_key_values = None,
                                    training = training
                        )

        if decoder_token_type_id is None:
            input_shape = shape_list(decoder_inputs)
            decoder_token_type_id = tf.fill(dims=input_shape, value=0)
        hidden_states = self.embedding_decoder(input_ids = decoder_inputs, token_type_ids = decoder_token_type_id)
        decoder_output = self.decoder(
            hidden_states = hidden_states,
            attention_mask = decoder_attention_mask,
            head_mask = head_mask,
            encoder_hidden_states = encoder_output,
            encoder_attention_mask = encoder_attention_mask,
            past_key_values = None,
            training = training,
        )

        logits = self.logit(decoder_output)
        start_logits, end_logits = tf.split(value=logits, num_or_size_splits=2, axis=-1)
        start_logits = self.start(tf.squeeze(input=start_logits, axis=-1))
        end_logits = self.end(tf.squeeze(input=end_logits, axis=-1))
        return start_logits, end_logits

In [70]:
config = Config()
config.input_vocab_size = tokenizer.vocab_size()
config.target_vocab_size = tokenizer.vocab_size()
model = TFRobertaModel(config, model_analys, 702)

In [71]:
checkpoint_path = "/content/drive/MyDrive/Khóa luận tốt nghiệp/QA/roberta_weight-final"

ckpt1 = tf.train.Checkpoint(model=model)
ckpt_manager1 = tf.train.CheckpointManager(ckpt1, checkpoint_path, max_to_keep=2)

if ckpt_manager1.latest_checkpoint:
  ckpt1.restore(ckpt_manager1.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!WARNING:tensorflow:Value in checkpoint could not be found in the restored object: (root).optimizer's state 'm' for (root).model.decoder.layer.4.bert_output.dense.bias



In [72]:
postag = getTypeOfWord(vncore, query)
postag = tf.convert_to_tensor([postag])
token_ipt = tokenizer(query, max_len = 277, truncation = True, padding = True)
token_opt = tokenizer(documents[np.argmax(scores)], max_len = 800, truncation = True, padding = True)
encoder_inputs = tf.convert_to_tensor([token_ipt['input_ids']])
encoder_attention_mask = tf.convert_to_tensor([token_ipt['attention_mask']])
decoder_inputs = tf.convert_to_tensor([token_opt['input_ids']])
decoder_attention_mask = tf.convert_to_tensor([token_opt['attention_mask']])

In [73]:
start, end = model(
    encoder_inputs = encoder_inputs,
    encoder_attention_mask = encoder_attention_mask,
    postag = postag,
    encoder_token_type_id = None,
    head_mask = None,
    decoder_inputs = decoder_inputs,
    decoder_attention_mask = decoder_attention_mask,
    decoder_token_type_id = None,
)

In [74]:
start = np.argmax(tf.nn.softmax(start, axis = -1).numpy()[0])
end = np.argmax(tf.nn.softmax(end, axis = -1).numpy()[0])

In [75]:
result = documents[np.argmax(scores)].split()[start:end+1]
' '.join(result)

'đào_tạo theo hệ_thống tín_chỉ là tập_hợp những quy_định về phương_thức đào_tạo thực_hiện theo hình_thức tích_luỹ tín_chỉ trong đó sinh_viên chủ_động lựa_chọn học từng học_phần tuân theo một_số ràng_buộc được quy_định trước nhắm tích_luỹ từng phần kiến_thức và tiến tới hoàn_thành toàn_bộ chương_trình đào_tạo để được cấp văn_bằng tốt_nghiệp'

In [76]:
start, end

(1, 48)